In [1]:
import os
import torch
import pandas as pd

model_pre_len = 100
model_seq_len = 300
model_tf_lr = 0.0001
model_batch = 64
model_feature_size=5
model_d_model=256
model_num_layers=1
model_dropout=0.1

# USE_MULTI_GPU = True
# # 设置默认的CUDA设备
# torch.cuda.set_device(0)
# # 初始化CUDA环境
# torch.cuda.init()
# if USE_MULTI_GPU and torch.cuda.device_count() > 1:
#     MULTI_GPU = True
#     os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#     os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5"  # 设置所有六张显卡的编号
#     device_ids = ['0','1','2','3','4','5',] # 设置所有六张显卡的编号
# else:
#     MULTI_GPU = False
#     device_ids = ['0']
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(MULTI_GPU)
# deviceCount = torch.cuda.device_count()
# torch.cuda.set_device(device)
# print(deviceCount)
# print(device)

volve_4 = './data/volve/volve_4.csv'
volve_5 = './data/volve/volve_5.csv'
volve_7 = './data/volve/volve_7.csv'
volve_9 = './data/volve/volve_9.csv'
volve_9A = './data/volve/volve_9A.csv'
volve_10 = './data/volve/volve_10.csv'
volve_12 = './data/volve/volve_12.csv'
volve_14 = './data/volve/volve_14.csv'
volve_15A = './data/volve/volve_15A.csv'
volve_4_5_7_9A_10 = './data/volve/volve_4_5_7_9A_10.csv'
volve_5_7_10_12 = './data/volve/volve_5_7_10_12.csv'
xj_3 =  './data/xj/well_3.csv'
xj_2 = './data/xj/well_2.csv'
xj_1 = './data/xj/well_1.csv'

bh_1 = './data/bh/bh_1.csv'
bh_2 = './data/bh/bh_2.csv'
bh_3 = './data/bh/bh_3.csv'
bh_4 = './data/bh/bh_4.csv'
bh_5 = './data/bh/bh_5.csv'
bh_6 = './data/bh/bh_6.csv'
bh_7 = './data/bh/bh_7.csv'
bh_8 = './data/bh/bh_8.csv'
bh_9 = './data/bh/bh_9.csv'
bh_10 = './data/bh/bh_10.csv'
bh_11 = './data/bh/bh_11.csv'
bh_12 = './data/bh/bh_12.csv'
bh_14 = './data/bh/bh_14.csv'
bh_15 = './data/bh/bh_15.csv'
bh_16 = './data/bh/bh_16csv'
bh_7_15 = './data/bh/bh_7_15.csv'
# 创建一个字典，其中包含所有客户端的训练和测试指标列表
loss_acc_r2_mse_mae_metrics = {
    'client_0': {
        'train': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        },
        'test': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        }
    },
    'client_1': {
        'train': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        },
        'test': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        }
    },
    'client_2': {
        'train': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        },
        'test': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        }
    },
    'server_model': {
        'test_volve': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        },
        'test_xj': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        },
        'test_bh': {
            'acc_size': [],'r2_size': [],'mse_size': [],'mae_size': [],'loss_size': []
        }
    }
}

de_t_p_metrics = {
    'client_0': {
        'train': {'depth': [],'true': [],'pre': []},
        'test': {'depth': [],'r2_size': [],'true': [],'pre': []}
    },
    'client_1': {
        'train': {'depth': [], 'true': [], 'pre': []},
        'test': {'depth': [], 'true': [], 'pre': []}
    },
    'client_2': {
        'train': {'depth': [], 'true': [], 'pre': []},
        'test': {'depth': [], 'true': [], 'pre': []}
    },
    'server_model': {
        'test_volve': {'depth': [], 'true': [], 'pre': []},
        'test_xj': {'depth': [], 'true': [], 'pre': []},
        'test_bh': {'depth': [], 'true': [], 'pre': []},
    },

}




In [2]:
import math
import warnings
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.optim import lr_scheduler
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from pylab import mpl
from utility_compare import *
warnings.filterwarnings('ignore')

# 设置matplotlib的配置
# mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体为黑体
mpl.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)  # 64*512
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # 64*1
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # 256   model/2
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        # pe.requires_grad = False
        self.register_buffer('pe', pe)  # 64*1*512

    def forward(self, x):  # [seq,batch,d_model]
        return x + self.pe[:x.size(0), :]  # 64*64*512

class TransAm(nn.Module):
    def __init__(self, feature_size=model_feature_size, d_model=model_d_model, num_layers=model_num_layers, dropout=model_dropout):
        super(TransAm, self).__init__()
        self.feature_size = feature_size
        self.model_type = 'Transformer'
        self.embedding = nn.Linear(feature_size, d_model)
        self.dec_input_fc = nn.Linear(feature_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)  # 50*512
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=8, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=8, dropout=dropout, batch_first=True)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)
        self.linear = nn.Linear(d_model, 1)
        self.src_mask = None
        self.src_key_padding_mask = None

    def forward(self, src,tgt,tgt_mask):

        # if self.src_key_padding_mask is None:
        #     mask_key = src_padding  # [batch,seq]
        #     self.src_key_padding_mask = mask_key
        src_em = self.embedding(src)  # [seq,batch,d_model]
        src_em_pos = self.pos_encoder(src_em)  # [seq,batch,d_model]
        encoder_output = self.transformer_encoder(src_em_pos)

        tgt_em = self.embedding(tgt)
        tgt_em_pos = self.pos_encoder(tgt_em)

        decoder_output = self.transformer_decoder(tgt_em_pos, encoder_output, tgt_mask=tgt_mask)

        output = self.linear(decoder_output)
        output_squeeze = output.squeeze()

        self.tgt_mask = None
        return output_squeeze


def data_load(path):

    data = pd.read_csv(path)
    # data = data[['MD', 'TVD', 'RPMA', 'WOBA', 'ROPA']]
   # data = data.iloc[::interval, :]

    # data = data.clip(lower=0)  # 设置小于0的数都赋0
    # data = data.apply(lambda x: x.mask((x < x.quantile(0.25) - 1.5 * (x.quantile(0.75) - x.quantile(0.25))) |
    #                                      (x > x.quantile(0.75) + 1.5 * (
    #                                                  x.quantile(0.75) - x.quantile(0.25)))).ffill().bfill())
    #
    # data = data.sort_values(by='MD')
    # data=data.reset_index(drop=True)
    data = data.astype('float32')
    data.dropna(inplace=True)
    data = data.values

    data_ =torch.tensor(data[:len(data)])
    maxc, _ = data_.max(dim=0)
    minc, _ = data_.min(dim=0)
    y_max = maxc[-1]
    y_min = minc[-1]
    de_max = maxc[0]
    de_min = minc[0]
    data_ = (data_ - minc) / (maxc - minc)

    data_last_index = data_.shape[0] - model_seq_len

    data_X = []
    data_Y = []
    for i in range(0, data_last_index - model_pre_len+1):
        data_x = np.expand_dims(data_[i:i + model_seq_len], 0)  # [1,seq,feature_size]
        data_y = np.expand_dims(data_[i + model_seq_len:i + model_seq_len + model_pre_len], 0)  # [1,seq,out_size]
        data_X.append(data_x)
        data_Y.append(data_y)

    data_X=np.concatenate(data_X, axis=0)
    data_Y = np.concatenate(data_Y, axis=0)

    process_data = torch.from_numpy(data_X).type(torch.float32)
    process_label = torch.from_numpy(data_Y).type(torch.float32)

    data_feature_size = process_data.shape[-1]

    dataset_train = TensorDataset(process_data, process_label)

    data_dataloader = DataLoader(dataset_train, batch_size=model_batch, shuffle=False)
    return data_dataloader,y_max,y_min, de_max,de_min

def averages(matrix):  # 计算平均值
    matrix = np.array(matrix)
    row_count, col_count = matrix.shape
    max_diagonal = row_count + col_count - 1
    diagonals = np.zeros(max_diagonal)
    counts = np.zeros(max_diagonal, dtype=int)
    for i in range(row_count):
        for j in range(col_count):
            num = matrix[i, j]
            diagonal_index = i + j
            diagonals[diagonal_index] += num
            counts[diagonal_index] += 1
    averages = diagonals / counts
    return averages

def acc_loss_plot_one(train_data, type_, path):
    plt.figure(figsize=(10, 5))
    plt.plot(train_data, label='train_data', color='blue', linewidth=1)
    plt.xlabel('epoch', fontsize=18)
    plt.title(f'train_test_{type_}')
    path_ = f'{path}'

    plt.grid()
    plt.savefig(path_)

    plt.legend()
   # plt.show()

def acc_loss_plot_two(train_data, test_data, type_, path):
    plt.figure(figsize=(10, 5))
    plt.plot(train_data, label='train_data', color='blue', linewidth=1)
    plt.plot(test_data, label='test_data', color='red', linewidth=1)
    plt.xlabel('epoch', fontsize=18)
    plt.title(f'train_test_{type_}')
    path_ = f'{path}'

    plt.grid()
    plt.savefig(path_)

    plt.legend()
  #  plt.show()

def true_test_plot(depth, true_data, predicted_data, type_, path):
    plt.figure(figsize=(20, 6))
    plt.plot(depth, true_data, label='true_data', color='blue', linewidth=1)
    plt.plot(depth, predicted_data, label='test_data', color='green', linewidth=1)
    plt.ylabel("GRA", fontsize=18)
    plt.xlabel('depth', fontsize=18)
    path_ = f'{path}'
    plt.grid()
    plt.savefig(path_)
#    plt.show()

In [ ]:
import torch.nn as nn
import torch.optim as optim

from datetime import datetime

USE_MULTI_GPU = True
# 设置默认的CUDA设备
torch.cuda.set_device(0)
# 初始化CUDA环境
torch.cuda.init()
if USE_MULTI_GPU and torch.cuda.device_count() > 1:
    MULTI_GPU = True
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5"  # 设置所有六张显卡的编号
    device_ids = ['0','1','2','3','4','5',] # 设置所有六张显卡的编号
else:
    MULTI_GPU = False
    device_ids = ['0']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(MULTI_GPU)
deviceCount = torch.cuda.device_count()
torch.cuda.set_device(device)
print(deviceCount)
print(device)


volve_train,volve_train_y_max,volve_train_y_min, volve_train_de_max,volve_train_de_min = data_load(bh_7_15)
volve_test,volve_test_y_max,volve_test_y_min, volve_test_de_max,volve_test_de_min =  data_load(bh_2)

model=TransAm().to(device)

criterion = nn.MSELoss()  # 占位符 索引为0.9


optimizer = optim.Adam(model.parameters(), lr=model_tf_lr, weight_decay=0.001)

# # 定义学习率衰减策略
# scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.2)


def train(TModel, loader,optimizer):
    epoch_loss = 0
    criterion = nn.MSELoss()  # 占位符 索引为0.9

    for X, y in loader:
        # X--[batch,seq,feature_size]  y--[batch,seq,feature_size]   64 300 13  64 50 13
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        mask = (torch.triu(torch.ones(y.size(1), y.size(1))) == 1).transpose(0, 1)
        tgt_mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)

        output = TModel(X, y, tgt_mask)
        loss = criterion(output, y[:, :, -1])
        loss.backward()
        torch.nn.utils.clip_grad_norm_(TModel.parameters(), 0.10)
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss

def test(TModel, tf_loader, y_max, y_min, de_max, de_min):
    epoch_loss = 0
    y_pre = []
    y_true = []
    y_depth = []
    criterion = nn.MSELoss()  # 占位符 索引为0.9
    for x, y in tf_loader:
        with torch.no_grad():
            label = y[:, :, -1].detach().view(1, len(y[:, :, -1]) * model_pre_len).squeeze()
            label = label * (y_max - y_min) + y_min
            label = label.numpy().tolist()
            y_true += label

            de = y[:, :, 0].detach().view(1, len(y[:, :, 0]) * model_pre_len).squeeze()
            de = de * (de_max - de_min) + de_min
            de = de.numpy().tolist()
            y_depth += de

            x, y = x.to(device), y.to(device)

            mask = (torch.triu(torch.ones(y.size(1), y.size(1))) == 1).transpose(0, 1)
            tgt_mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)

            output = TModel(x, y, tgt_mask)

            loss = criterion(output, y[:, :, -1])
            epoch_loss += loss.item()

            hat = output.cpu().detach().view(1, len(y[:, :, -1]) * model_pre_len).squeeze()
            hat = hat * (y_max - y_min) + y_min
            hat = hat.numpy().tolist()
            y_pre += hat

    label = np.array(y_true)
    predict = np.array(y_pre)
    dep = np.array(y_depth)

    seq_label = label.reshape(int(len(label) / model_pre_len), model_pre_len)
    seq_predict = predict.reshape(int(len(predict) / model_pre_len), model_pre_len)
    seq_depth = dep.reshape(int(len(dep) / model_pre_len), model_pre_len)

    true = np.concatenate((seq_label[:-1, 0], seq_label[-1, :]), axis=0)
    depth = np.concatenate((seq_depth[:-1, 0], seq_depth[-1, :]), axis=0)
    pre = averages(seq_predict)

    r2 = r2_score(true, pre)
    acc = 1 - (np.abs(pre - true) / (true + 1e-8)).mean()
    mse = mean_squared_error(true, pre)

    mae = mean_absolute_error(true, pre)

    return acc, r2, mse, mae, epoch_loss, true, pre, depth

def initiate():
    train_acc_size = []
    train_r2_size = []
    train_mse_size = []
    train_mae_size = []
    train_loss_size = []

    test_acc_size = []
    test_r2_size = []
    test_mse_size = []
    test_mae_size = []
    test_loss_size = []
    total_acc = 0
    for epoch in range(100):
        start = datetime.now()
        model.train()
        train(model, volve_train,optimizer)
        model.eval()
        train_acc, train_r2, train_mse, train_mae, train_loss, true_train, pre_train, train_depth = test(model,
                                                                                                         volve_train,
                                                                                                         volve_train_y_max, volve_train_y_min,
                                                                                                         volve_train_de_max, volve_train_de_min)
        train_mse_size.append(train_mse)
        train_mae_size.append(train_mae)
        train_acc_size.append(train_acc)
        train_r2_size.append(train_r2)
        train_loss_size.append(train_loss)
        print('Epoch:', 'train %04d' % epoch, 'loss =', '{:.6f}'.format(train_loss), ' acc =',
              '{:.6f}'.format(train_acc), ' r2 =', '{:.6f}'.format(train_r2),
              ' mse =', '{:.6f}'.format(train_mse), ' mae =', '{:.6f}'.format(train_mae), 'time = ', start)

        test_acc, test_r2, test_mse, test_mae, test_loss, true_test, pre_test, test_depth = test(model, volve_test,
                                                                                                 volve_test_y_max, volve_test_y_min,
                                                                                                 volve_test_de_max, volve_test_de_min)
        test_mse_size.append(test_mse)
        test_mae_size.append(test_mae)
        test_acc_size.append(test_acc)
        test_r2_size.append(test_r2)
        test_loss_size.append(test_loss)
        print('Epoch:', 'test %04d' % epoch, 'loss =', '{:.6f}'.format(test_loss), ' acc =', '{:.6f}'.format(test_acc),
              ' r2 =', '{:.6f}'.format(test_r2),
              ' mse =', '{:.6f}'.format(test_mse), ' mae =', '{:.6f}'.format(test_mae), 'time = ', start)

        loss_acc_mse_mae_dict = {'train_loss': train_loss_size, 'test_loss': test_loss_size,
                                 'train_acc': train_acc_size, 'test_acc': test_acc_size,
                                 'train_r2': train_r2_size, 'test_r2': test_r2_size,
                                 'train_mse': train_mse_size, 'train_mae': train_mae_size,
                                 'test_mse': test_mse_size, 'test_mae': test_mae_size, }
        loss_acc_mse_mae = pd.DataFrame(loss_acc_mse_mae_dict)

        train_de = pd.DataFrame(train_depth, columns=['train_depth'])
        train_t = pd.DataFrame(true_train, columns=['train_true'])
        train_p = pd.DataFrame(pre_train, columns=['train_pre'])
        test_de = pd.DataFrame(test_depth, columns=['test_depth'])
        test_t = pd.DataFrame(true_test, columns=['test_true'])
        test_p = pd.DataFrame(pre_test, columns=['test_pre'])

        csv_train = pd.concat([train_de, train_t, train_p], axis=1)
        csv_test = pd.concat([test_de, test_t, test_p], axis=1)

        loss_acc_mse_mae.to_csv('./compare/bh1/out/loss_acc_mse_mae_.csv', sep=",", index=True)

        torch.save(model.state_dict(), './compare/bh1/model/Model_volve.pkl')

        csv_train.to_csv('./compare/bh1/out/rel_pre_train.csv', sep=",", index=True)
        csv_test.to_csv('./compare/bh1/out/rel_pre_test.csv', sep=",", index=True)

        acc_loss_plot_two(loss_acc_mse_mae['train_loss'], loss_acc_mse_mae['test_loss'], 'loss',
                      './compare/bh1/out/loss.png')
        acc_loss_plot_two(loss_acc_mse_mae['train_r2'], loss_acc_mse_mae['test_r2'], 'r2',
                      './compare/bh1/out/acc.png')
        true_test_plot(csv_test['test_depth'], csv_test['test_true'], csv_test['test_pre'], 'test',
                       './compare/bh1/out/pre_true_test.png')
        true_test_plot(csv_train['train_depth'], csv_train['train_true'], csv_train['train_pre'], 'train',
                        './compare/bh1/out/pre_true_train.png')


initiate()

False
1
cuda:0
Epoch: train 0000 loss = 27.933239  acc = 0.197876  r2 = -0.436275  mse = 84.922292  mae = 8.175885 time =  2025-02-18 09:22:28.538324
Epoch: test 0000 loss = 7.599883  acc = 0.370613  r2 = -1.324057  mse = 29.795908  mae = 5.046198 time =  2025-02-18 09:22:28.538324
Epoch: train 0001 loss = 30.732374  acc = 0.420232  r2 = -0.585276  mse = 93.732199  mae = 7.946077 time =  2025-02-18 09:25:17.147501
Epoch: test 0001 loss = 8.340499  acc = 0.413842  r2 = -1.553466  mse = 32.737077  mae = 5.067267 time =  2025-02-18 09:25:17.147501
Epoch: train 0002 loss = 14.411885  acc = 0.432292  r2 = 0.259086  mse = 43.807841  mae = 5.840120 time =  2025-02-18 09:28:06.023568
Epoch: test 0002 loss = 3.933378  acc = 0.559199  r2 = -0.202270  mse = 15.413877  mae = 3.594699 time =  2025-02-18 09:28:06.023568
Epoch: train 0003 loss = 0.107105  acc = 0.967462  r2 = 0.994940  mse = 0.299157  mae = 0.431964 time =  2025-02-18 09:30:54.771781
Epoch: test 0003 loss = 0.021775  acc = 0.971075  

Epoch: test 0031 loss = 0.078507  acc = 0.937659  r2 = 0.977406  mse = 0.289668  mae = 0.450740 time =  2025-02-18 10:49:39.297688
Epoch: train 0032 loss = 1.226216  acc = 0.795639  r2 = 0.937680  mse = 3.684753  mae = 1.782634 time =  2025-02-18 10:52:27.682100
Epoch: test 0032 loss = 0.184641  acc = 0.876835  r2 = 0.944373  mse = 0.713168  mae = 0.759069 time =  2025-02-18 10:52:27.682100
Epoch: train 0033 loss = 16.007880  acc = 0.298313  r2 = 0.178257  mse = 48.586970  mae = 6.468419 time =  2025-02-18 10:55:16.081049
Epoch: test 0033 loss = 3.695884  acc = 0.503652  r2 = -0.132119  mse = 14.514498  mae = 3.676454 time =  2025-02-18 10:55:16.081049
Epoch: train 0034 loss = 0.540485  acc = 0.882130  r2 = 0.973297  mse = 1.578847  mae = 1.122984 time =  2025-02-18 10:58:04.114288
Epoch: test 0034 loss = 0.081107  acc = 0.926351  r2 = 0.976884  mse = 0.296362  mae = 0.481264 time =  2025-02-18 10:58:04.114288
Epoch: train 0035 loss = 1.018910  acc = 0.840178  r2 = 0.948614  mse = 3.03

Epoch: train 0063 loss = 8.651598  acc = 0.577984  r2 = 0.570418  mse = 25.399805  mae = 4.507401 time =  2025-02-18 12:19:33.976062
Epoch: test 0063 loss = 2.072778  acc = 0.658821  r2 = 0.388023  mse = 7.845935  mae = 2.645356 time =  2025-02-18 12:19:33.976062
Epoch: train 0064 loss = 0.504997  acc = 0.878407  r2 = 0.977828  mse = 1.310966  mae = 1.061397 time =  2025-02-18 12:22:22.792119
Epoch: test 0064 loss = 0.096917  acc = 0.920182  r2 = 0.975429  mse = 0.315010  mae = 0.534255 time =  2025-02-18 12:22:22.792119
Epoch: train 0065 loss = 0.296360  acc = 0.923236  r2 = 0.986920  mse = 0.773375  mae = 0.768173 time =  2025-02-18 12:25:11.400292
Epoch: test 0065 loss = 0.051852  acc = 0.949545  r2 = 0.987138  mse = 0.164903  mae = 0.366901 time =  2025-02-18 12:25:11.400292
Epoch: train 0066 loss = 0.338281  acc = 0.913321  r2 = 0.985125  mse = 0.879535  mae = 0.851254 time =  2025-02-18 12:27:59.590163
Epoch: test 0066 loss = 0.056020  acc = 0.941627  r2 = 0.986502  mse = 0.17305